In [2]:
!pip install transformers
!pip install spacy sklearn
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 4.0 MB 34.7 MB/s 
     |████████████████████████████████| 596 kB 62.5 MB/s 
     |████████████████████████████████| 895 kB 67.1 MB/s 
     |████████████████████████████████| 6.6 MB 62.3 MB/s 
     |████████████████████████████████| 77 kB 8.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 12.0 MB 31.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [3]:
!git clone https://github.com/nishithsharma9/NLUProject.git

Cloning into 'NLUProject'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 102 (delta 28), reused 80 (delta 15), pack-reused 0
Receiving objects: 100% (102/102), 96.75 MiB | 8.84 MiB/s, done.
Resolving deltas: 100% (28/28), done.
Checking out files: 100% (44/44), done.


In [4]:
import spacy
from spacy.util import minibatch
import pandas as pd
import random

text_data = pd.read_csv('/content/NLUProject/Data/BiasPredictionScores/test.csv')

In [5]:
toxicClassifierModelSpacy = spacy.load("/content/NLUProject/Transformer/Models/ToxicTextClassifierModel/model")

In [6]:
toxicSpanModelSpacy = spacy.load("/content/NLUProject/Transformer/Models/ToxicTextSpanDetectionModel/model")

In [7]:
from transformers import pipeline

toxicity_classifier = pipeline(
     'text-classification', 
     model='unitary/toxic-bert', 
     tokenizer='bert-base-uncased', 
     function_to_apply='sigmoid', 
     return_all_scores=True,
     device = 0
     )

Downloading:   0%|          | 0.00/811 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
toxicity_classifier.device

device(type='cuda', index=0)

In [8]:
import pandas as pd
input_test_data = pd.read_csv('/content/NLUProject/Data/ToxicTextClassifierData/test.csv')

In [9]:
input_test_data.count()

id              153164
comment_text    153164
dtype: int64

In [10]:
text = input_test_data.iloc[293].comment_text
text

'" \n\n No kidding  right up there with the ""Superman"" and ""Poltergeist"" curses.  If I was Samuel L. Jackson, my ass would be on lockdown right about now.   "'

In [11]:
toxicity_classifier(text)

[[{'label': 'toxic', 'score': 0.8300319910049438},
  {'label': 'severe_toxic', 'score': 0.013729806989431381},
  {'label': 'obscene', 'score': 0.7672582864761353},
  {'label': 'threat', 'score': 0.004491507075726986},
  {'label': 'insult', 'score': 0.06659699976444244},
  {'label': 'identity_hate', 'score': 0.0031061989720910788}]]

In [12]:
doc1 = toxicClassifierModelSpacy(text)
doc2 = toxicSpanModelSpacy(text)

In [14]:
print(doc1.cats,doc2.ents)
print(str(doc2.ents))

{'POSITIVE': 0.42392611503601074, 'NEGETIVE': 0.5760738849639893} (ass,)
(ass,)


In [15]:
doc1.cats['POSITIVE']<0.5

True

In [16]:
for ent in doc2.ents:
  text = text.replace(str(ent),"[MASK]")

In [17]:
text

'" \n\n No kidding  right up there with the ""Superman"" and ""Poltergeist"" curses.  If I was Samuel L. Jackson, my [MASK] would be on lockdown right about now.   "'

In [18]:
toxicity_classifier(text)

[[{'label': 'toxic', 'score': 0.005974349100142717},
  {'label': 'severe_toxic', 'score': 0.00012137035810155794},
  {'label': 'obscene', 'score': 0.0004806668439414352},
  {'label': 'threat', 'score': 0.00021033122902736068},
  {'label': 'insult', 'score': 0.00036538008134812117},
  {'label': 'identity_hate', 'score': 0.00022773040109314024}]]

In [19]:
output_labels = toxicity_classifier('you are an idiot')

In [20]:
columns = set(['id','comment_text'])
for label_row in output_labels[0]:
  columns.add('prediction_'+label_row['label'])

In [ ]:
import pandas as pd
count = 8
start,end = 70000,80000
max = 153164
while(start<max):
  if end>max:
    end = max-1
  input_internval=input_test_data[start:end]
  output_pred_df = pd.DataFrame(columns = columns)
  error_df = pd.DataFrame(columns = {'id','comment_text'})
  dummy = pd.DataFrame(columns = {'id','comment_text','ents'})
  for index, row in input_internval.iterrows():
    id, comment_text= row.values

    r = {'id':row.id, 'comment_text':row.comment_text,
         'prediction_toxic':0,
         'prediction_severe_toxic':0,
         'prediction_threat':0,	
         'prediction_obscene':0,	
         'prediction_insult':0,
         'prediction_identity_hate':0}

    doc1 = toxicClassifierModelSpacy(comment_text)
    if doc1.cats['POSITIVE']<0.5:
      try:
        
        doc2 = toxicSpanModelSpacy(comment_text)
        for ent in doc2.ents:
          comment_text = comment_text.replace(str(ent),'[MASK]')
        r['comment_text']=comment_text
        dummy.append({'id':id,'comment_text':comment_text,'ents':str(doc2.ents)},ignore_index=True)
        output_labels = toxicity_classifier(comment_text)
        for i in output_labels[0]: r['prediction_'+i['label']]=i['score']
        output_pred_df = output_pred_df.append(r,ignore_index=True)
      except:
        error_df = error_df.append(r,ignore_index=True)
    else:
      output_pred_df = output_pred_df.append(r,ignore_index=True)
  output_pred_df.to_csv('/content/drive/MyDrive/NLUProject/outputPred'+str(count)+'.csv')
  dummy.to_csv('/content/drive/MyDrive/NLUProject/dummy'+str(count)+'.csv')
  print(count,start,end)
  start +=10000
  end+=10000
  count+=1

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:998: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


8 70000 80000
9 80000 90000
10 90000 100000
11 100000 110000
12 110000 120000
13 120000 130000
14 130000 140000
15 140000 150000
16 150000 153163


In [ ]:
import pandas as pd
dummy = pd.DataFrame(columns = {'id','comment_text','ents'})

In [ ]:
dummy.append({'id':1,'comment_text':"asda",'ents':"asda"},ignore_index=True)

,comment_text,ents,id
0,asda,asda,1


In [ ]:
t = pd.read_csv('test')

In [ ]:
t[t['id']=='0013fed3aeae76b7']

,Unnamed: 0,id,prediction_obscene,comment_text,prediction_threat,prediction_identity_hate,prediction_insult,prediction_severe_toxic,prediction_toxic
47,47,0013fed3aeae76b7,0.944201,DJ Robinson is gay as hell! he sucks his [MASK...,0.033355,0.895092,0.933365,0.469912,0.996636


In [22]:
import pandas as pd

output_pred_df = pd.DataFrame(columns = columns)

for count in range(0,16):
  output_pred_df = output_pred_df.append(pd.read_csv('/content/drive/MyDrive/NLUProject/outputPred'+str(count)+'.csv'), ignore_index=True)



In [43]:
ind = 7
print(output_pred_df.iloc[ind].comment_text)

output_pred_df.iloc[ind]

:Dear god this site is horrible.


prediction_insult                                   0.006455
prediction_toxic                                    0.474604
prediction_obscene                                  0.006783
prediction_severe_toxic                              0.00059
prediction_threat                                   0.001611
comment_text                :Dear god this site is horrible.
prediction_identity_hate                            0.001572
id                                          000247e83dcc1211
Unnamed: 0                                               7.0
Name: 7, dtype: object

In [44]:
output_pred_df.to_csv("/content/drive/MyDrive/NLUProject/outputPredMasked.csv")

In [46]:
output_pred_df.columns

Index(['prediction_insult', 'prediction_toxic', 'prediction_obscene',
       'prediction_severe_toxic', 'prediction_threat', 'comment_text',
       'prediction_identity_hate', 'id', 'Unnamed: 0'],
      dtype='object')

In [47]:
output_pred_df_new = output_pred_df.drop(['comment_text'], axis=1)

In [49]:
output_pred_df_new.to_csv("/content/drive/MyDrive/NLUProject/outputPredMaskedNoText.csv")